In [8]:
import sqlite3
import numpy as np
import mediapipe as mp

# SQLite 데이터베이스 초기화
def init_db():
    conn = sqlite3.connect('face_access_control.db')
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS faces (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        name TEXT,
                        landmarks TEXT)''')
    conn.commit()
    conn.close()

# 얼굴 랜드마크 DB에 저장
def save_landmarks_to_db(name, landmarks):
    conn = sqlite3.connect('face_access_control.db')
    cursor = conn.cursor()
    
    # 랜드마크를 문자열로 변환 (튜플 목록을 문자열로 변환)
    landmarks_str = str(landmarks)
    cursor.execute("INSERT INTO faces (name, landmarks) VALUES (?, ?)", (name, landmarks_str))
    conn.commit()
    conn.close()

# DB에서 랜드마크 가져오기
def get_landmarks_from_db(name):
    conn = sqlite3.connect('face_access_control.db')  # 파일 이름 수정
    cursor = conn.cursor()
    cursor.execute("SELECT landmarks FROM faces WHERE name=?", (name,))
    result = cursor.fetchone()
    conn.close()
    
    if result:
        landmarks_str = result[0]
        landmarks = eval(landmarks_str)  # 문자열을 리스트로 변환
        return landmarks
    return None


In [10]:
# 얼굴 인식 및 랜드마크 추출 후 저장
import cv2
import mediapipe as mp

# MediaPipe 설정
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

# 얼굴 랜드마크 추출
def extract_face_landmarks(frame):
    with mp_face_mesh.FaceMesh(min_detection_confidence=0.2, min_tracking_confidence=0.2) as face_mesh:
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = face_mesh.process(rgb_frame)
        
        if result.multi_face_landmarks:
            landmarks = []
            for landmarks_list in result.multi_face_landmarks:
                for landmark in landmarks_list.landmark:
                    x = int(landmark.x * frame.shape[1])
                    y = int(landmark.y * frame.shape[0])
                    landmarks.append((x, y))
            return landmarks
    return None

# 얼굴을 인식하고 랜드마크 저장
def capture_and_save_landmarks():
    name = input("얼굴 인증을 위한 이름을 입력하세요: ")
    
    cap = cv2.VideoCapture(0)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            continue

        landmarks = extract_face_landmarks(frame)
        if landmarks:
            # 랜드마크 추출 후 DB에 저장
            save_landmarks_to_db(name, landmarks)
            print("얼굴 랜드마크가 저장되었습니다.")
            break

        # 얼굴 랜드마크를 화면에 표시
        cv2.imshow("Face Recognition", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# DB 초기화
init_db()

# 랜드마크 저장 테스트
capture_and_save_landmarks() # 경로 지정


얼굴 랜드마크가 저장되었습니다.


In [11]:
# 두 랜드마크 리스트 간의 유사도 계산 (간단한 L2 거리 사용)
def calculate_similarity(landmarks1, landmarks2):
    if len(landmarks1) != len(landmarks2):
        return float('inf')  # 길이가 다르면 비교 불가
    
    diff = np.linalg.norm(np.array(landmarks1) - np.array(landmarks2), axis=1)
    return np.mean(diff)

# 얼굴 인증 함수
def verify_face():
    name = input("인증할 이름을 입력하세요: ")
    
    # DB에서 저장된 랜드마크 가져오기
    saved_landmarks = get_landmarks_from_db(name)
    if not saved_landmarks:
        print("해당 이름으로 저장된 얼굴 랜드마크가 없습니다.")
        return
    
    print("카메라를 통해 얼굴을 캡처하고 있습니다...")
    cap = cv2.VideoCapture(0)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            continue

        # 실시간 얼굴 랜드마크 추출
        current_landmarks = extract_face_landmarks(frame)
        if current_landmarks:
            # 랜드마크 비교
            similarity = calculate_similarity(saved_landmarks, current_landmarks)
            print(f"유사도: {similarity}")

            # 유사도가 임계값 이하인 경우 인증 성공
            if similarity < 120:  # 유사도 임계값 설정 (조정 필요)
                print(f"인증 성공: {name}")
                break
            else:
                print("인증 실패: 유사도가 낮습니다.")

        # 얼굴 랜드마크를 화면에 표시
        cv2.imshow("Face Verification", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("인증을 취소했습니다.")
            break

    cap.release()
    cv2.destroyAllWindows()

# 얼굴 인증 실행
verify_face()


카메라를 통해 얼굴을 캡처하고 있습니다...
유사도: 4.7998831789994965
인증 성공: seok
